In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from glob import glob
import os
import json 
import random
from xgboost import XGBClassifier

In [3]:
path = os.getcwd()

In [4]:
# 분석에 사용할 feature 선택
csv_features = ['내부 온도 1 평균', '내부 온도 1 최고', '내부 온도 1 최저', '내부 습도 1 평균', '내부 습도 1 최고', 
                '내부 습도 1 최저', '내부 이슬점 평균', '내부 이슬점 최고', '내부 이슬점 최저']

# csv_files = sorted(glob('sample_data/*/*.csv'))
csv_files = sorted(glob(path + '\\sample_data\\sample_data\\*\\*.csv'))

temp_csv = pd.read_csv(csv_files[0])[csv_features]
max_arr, min_arr = temp_csv.max().to_numpy(), temp_csv.min().to_numpy()

# feature 별 최대값, 최솟값 계산
for csv in tqdm(csv_files[1:]):
    temp_csv = pd.read_csv(csv)[csv_features]
    temp_max, temp_min = temp_csv.max().to_numpy(), temp_csv.min().to_numpy()
    max_arr = np.max([max_arr,temp_max], axis=0)
    min_arr = np.min([min_arr,temp_min], axis=0)

# feature 별 최대값, 최솟값 dictionary 생성
csv_feature_dict = {csv_features[i]:[min_arr[i], max_arr[i]] for i in range(len(csv_features))}
csv_feature_dict

100%|██████████| 498/498 [00:02<00:00, 242.90it/s]


{'내부 온도 1 평균': [14.4, 47.3],
 '내부 온도 1 최고': [14.5, 47.6],
 '내부 온도 1 최저': [14.4, 47.0],
 '내부 습도 1 평균': [34.1, 100.0],
 '내부 습도 1 최고': [36.5, 100.0],
 '내부 습도 1 최저': [32.4, 100.0],
 '내부 이슬점 평균': [12.4, 29.9],
 '내부 이슬점 최고': [12.8, 31.9],
 '내부 이슬점 최저': [12.1, 29.1]}

In [5]:
# 제공된 sample data는 파프리카와 시설포도 2종류의 작물만 존재
label_description = {
 '3_00_0': '파프리카_정상',
 '3_a9_1': '파프리카흰가루병_초기',
 '3_a9_2': '파프리카흰가루병_중기',
 '3_a9_3': '파프리카흰가루병_말기',
 '3_a10_1': '파프리카잘록병_초기',
 '3_a10_2': '파프리카잘록병_중기',
 '3_a10_3': '파프리카잘록병_말기',
 '3_b3_1': '칼슘결핍_초기',
 '3_b3_2': '칼슘결핍_중기',
 '3_b3_3': '칼슘결핍_말기',
 '3_b6_1': '다량원소결핍 (N)_초기',
 '3_b6_2': '다량원소결핍 (N)_중기',
 '3_b6_3': '다량원소결핍 (N)_말기',
 '3_b7_1': '다량원소결핍 (P)_초기',
 '3_b7_2': '다량원소결핍 (P)_중기',
 '3_b7_3': '다량원소결핍 (P)_말기',
 '3_b8_1': '다량원소결핍 (K)_초기',
 '3_b8_2': '다량원소결핍 (K)_중기',
 '3_b8_3': '다량원소결핍 (K)_말기',
 '6_00_0': '시설포도_정상',
 '6_a11_1': '시설포도탄저병_초기',
 '6_a11_2': '시설포도탄저병_중기',
 '6_a11_3': '시설포도탄저병_말기',
 '6_a12_1': '시설포도노균병_초기',
 '6_a12_2': '시설포도노균병_중기',
 '6_a12_3': '시설포도노균병_말기',
 '6_b4_1': '일소피해_초기',
 '6_b4_2': '일소피해_중기',
 '6_b4_3': '일소피해_말기',
 '6_b5_1': '축과병_초기',
 '6_b5_2': '축과병_중기',
 '6_b5_3': '축과병_말기',
}

label_encoder = {key:idx for idx, key in enumerate(label_description)}
label_decoder = {val:key for key, val in label_encoder.items()}

In [6]:
class DataController():
    def __init__(self,csvfeatures,csvfeaturedict):
        self.csv_features = csvfeatures
        self.csv_feature_dict = csvfeaturedict
    
    def road_csv(self,foldnam,timenum):
        df = pd.read_csv(foldnam)
        return df.loc[:timenum-1,self.csv_features]      #csv파일 제일 짧은게 291개임 오류인지는 모르겠으나 일단 최소길이로 통일하여 처리
    
    def scaling(self,minmaxdic,df):
        for col in minmaxdic.keys():
            df.loc[:,col] = df.loc[:,col] - csv_feature_dict[col][0]
            df.loc[:,col] = df.loc[:,col] / (csv_feature_dict[col][1]-csv_feature_dict[col][0])
        return df
    
    
    def getimage(self,imgpath):
        img = cv2.imread(imgpath)
        img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
        img = img.astype(np.float32)/223  ##픽셀값을 0~1사이로 정규화
        # img = np.transpose(img, (2,0,1))
        return img
    
    def getlable(self,jsonpath):
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)

        crop = json_file['annotations']['crop']
        disease = json_file['annotations']['disease']
        risk = json_file['annotations']['risk']
        label = f'{crop}_{disease}_{risk}'
        return label
    
    def getdata(self,datapath,timenum,featnum):

        csvarr = np.empty((0,timenum,featnum), float)
        imgarr = np.empty((0,224,224,3), float)
        lablearr = np.array([],dtype = int)
        
        # predictor = np.append(predictor,ndf.reshape(-1,timenum,featnum),axis = 0)
        
        for ind,i in enumerate(datapath):
            
            if glob(i + '/*.csv') == []:  #10462폴더 비어있음. 다음에 확인해보기
                pass
            else:
                csvpath = glob(i + '/*.csv')[0]
                imgpath = glob(i + '/*.jpg')[0]
                jsonpath = glob(i + '/*.json')[0]
                # con = DataController()
                df = self.road_csv(csvpath,timenum)
                df2 = self.scaling(csv_feature_dict,df).to_numpy().reshape(-1,timenum,featnum)
                imgdata = self.getimage(imgpath).reshape(-1,224,224,3)
                label = int(label_encoder[self.getlable(jsonpath)])
                # label = self.getlable(jsonpath)
                
                
                csvarr = np.append(csvarr,df2, axis = 0)
                imgarr = np.append(imgarr,imgdata, axis = 0)
                lablearr = np.append(lablearr,label)
            
        return [csvarr,imgarr],lablearr
            
        
        
        
          

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.resnet import ResNet50

In [8]:
#프로젝트에 있는 모든 데이터폴터 불러오기
data_files = glob(path + '\\sample_data\\sample_data\\*')
#셔플
random.shuffle(data_files)
#앞에서 300번째까지 트레인셋으로
trainfiles = data_files[:400]
#나머지는 테스트셋으로
testfiles = data_files[400:]

# 데이터 컨트롤러
dacon = DataController(csv_features,csv_feature_dict)
# 배치화된 데이터셋 만들기
# test셋용으로는 train = False로 하여 배치안생성하게됨
x_train,y_train = dacon.getdata(trainfiles,260,9)
x_test,y_test = dacon.getdata(testfiles,260,9)


In [9]:
class preprmodel(keras.Model):
    def __init__(self, imgmodel,name = None):
        super(preprmodel, self).__init__()
        self.imgmodel = imgmodel
        
        self.block1_layer1 = tf.keras.layers.LSTM(128)
        self.block1_layer2 = tf.keras.layers.Dense(64)
        self.block1_layer3 = tf.keras.layers.Dense(32)
        



    def call(self, inputs):
        x1 = self.imgmodel(inputs[1])

        x2 = self.block1_layer1(inputs[0])
        x2 = self.block1_layer2(x2)
        x2 = self.block1_layer3(x2)
    
        x = tf.concat([x1,x2],axis=1)
        return(x)

model_RESNET50 = ResNet50(weights='imagenet')
prepromodel = preprmodel(model_RESNET50)

x_prime_train = prepromodel(x_train)
x_prime_test = prepromodel(x_test)

In [10]:
y_train


array([ 0, 19, 20, 19, 19,  0,  0, 19, 19, 24,  1,  2, 19,  2,  0,  2, 19,
        0,  0,  0, 19, 19, 19,  2, 19, 19,  0, 19,  1,  0,  0,  0,  3, 19,
        0,  0,  0,  2, 26,  2, 19,  0, 19,  2, 19, 19, 24,  2,  1, 19,  1,
        0,  2, 19, 19,  1, 19, 19, 19, 19,  1, 19, 19,  1,  0, 23,  0, 19,
        1, 19,  0, 19,  0, 19, 19,  0,  0,  1, 19, 19, 19, 19,  0,  2,  2,
        1,  3, 19, 19, 19,  2,  1,  0, 19,  0, 19,  0,  0, 19,  0,  0,  0,
       19, 20,  0, 24, 24,  1, 19, 19, 19,  2,  0, 19, 19,  1, 24, 19,  0,
       19,  0,  0, 19,  0, 19, 19,  0,  0, 19, 19,  6, 19,  0,  0, 19,  0,
        1, 19,  0,  0,  1,  0,  0,  1, 19,  1, 19, 19,  0,  1,  0, 19, 19,
       24,  0, 19, 19, 19,  0,  0,  2, 19,  0, 19, 19, 19, 26, 19,  0, 19,
        0,  0, 19, 19,  0, 19,  0,  0,  6, 19, 19, 19, 19,  0, 29,  0,  0,
       19, 19,  0, 19,  1,  0,  1,  0, 19, 19,  0,  0, 19,  1,  0, 24, 19,
        0,  2,  0, 19, 19,  2,  0, 19,  0,  0, 19, 19,  3, 19, 21, 19, 19,
       19,  0,  2,  4, 19

In [26]:
# **최적화파트**
import pickle
from hyperopt import fmin, tpe, hp, Trials, space_eval, STATUS_OK
from sklearn.metrics import f1_score
import time
# # f1_score(y_true, y_pred, average=[‘micro’, ‘macro’, ‘samples’,’weighted’ 중 하나 선택])
# f1_score(y_test, y_pred, average='macro')

###서치공간
space_xgb = [hp.uniform('learn',0.01,0.8),            #학습률
         hp.quniform('nesti',100,350,q=10),          #
         hp.quniform('maxd',3,8,q=1),                #
         hp.quniform('minc',  1,10,q=1),             #과적합을 방지할 목적으로사용한다. 기본값은 1이지만 너무 높은값은 오히려 과소적합을 일으키기 때문에 적절한 값을 찾아야함 
         hp.uniform('gamm',  0.1,3),              #
         hp.uniform('subsa',  0.5,1),
         hp.uniform('colsample_b',  0.6,0.9),
         #hp.quniform('scalepos',  0.8,1.2,q=0.1)
         #hp.quniform('times',  0,6,q=1)
         ]


###함수
def XGB(args):
    learn,nesti,maxd,minc,gamm,subsa,colsample_b = args
    nesti,maxd,minc=int(nesti),int(maxd),int(minc)#,int(times)
    xgb=XGBClassifier(
        learning_rate =learn,
        n_estimators=nesti,
        max_depth=maxd,
        min_child_weight=minc,
        gamma=gamm,
        subsample=subsa,
        colsample_bytree=colsample_b,
        # scale_pos_weight=scalepos,
        #seed=seed_value,
        use_label_encoder=False,
        n_jobs = -1,
        seed = 100,
        # num_class=32,
        )
    
    for t in range(len(Train_set_X)):
        x_train, y_train = Train_set_X[t], Train_set_y[t]
        x_val, y_val = Val_set_X[t], Val_set_y[t]
        xgb.fit(x_train, y_train.ravel()) #validation_data=(x_val, y_val))
        y_pred=xgb.predict(x_val)
        y_pred=y_pred.reshape((len(y_pred),1),)


        result = -f1_score(y_val, y_pred, average='macro')
        resultdf.loc[t,['result']]=result
        
        del x_train, y_train
        del x_val, y_val
    #print(resultdf['auc'])
    result = resultdf['result'].mean()
    return(result)

from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
kf = RepeatedKFold(n_splits=5, n_repeats=5,random_state = 100)
# kf = StratifiedKFold()
Train_set_X = []
Train_set_y = []
Val_set_X = []
Val_set_y = []
#
x_prime_train = np.array(x_prime_train)
x_prime_test = np.array(x_prime_test)

#
for train_index, val_index in kf.split(x_prime_train,y_train):
    print("TRAIN:", train_index, "Val:", val_index)
    X_tra, X_val = x_prime_train[train_index,:], x_prime_train[val_index,:]
    y_tra, y_val = y_train[train_index], y_train[val_index]

    Train_set_X.append(X_tra)
    Train_set_y.append(y_tra)
    Val_set_X.append(X_val)
    Val_set_y.append(y_val)

Train_set_X = np.array(Train_set_X)
Train_set_y = np.array(Train_set_y)
Val_set_X = np.array(Val_set_X)
Val_set_y = np.array(Val_set_y)
#
result = {"result":[np.nan]}
resultdf = pd.DataFrame(result)

TRAIN: [  0   2   3   4   6   8   9  10  11  12  13  14  15  16  17  19  20  22
  24  25  26  27  29  30  31  32  34  35  36  37  38  39  40  41  42  44
  45  46  47  48  49  50  51  52  53  55  57  58  59  60  61  62  63  65
  66  67  68  70  71  72  73  74  76  77  79  80  81  82  83  84  85  86
  87  88  89  90  91  92  93  94  95  99 100 101 102 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 119 120 121 122 123 126 128 130 131
 132 133 135 137 138 139 141 142 143 144 145 146 147 148 150 152 154 155
 156 157 159 160 161 162 163 164 165 167 168 170 171 172 175 176 177 179
 180 181 182 183 185 186 188 189 190 191 192 193 194 195 196 197 199 200
 202 203 204 207 209 210 211 212 213 214 215 216 218 219 220 221 222 223
 224 226 227 228 229 230 232 233 234 237 238 239 240 241 243 245 246 247
 249 250 251 252 253 254 255 256 257 259 261 262 264 265 266 268 269 270
 273 274 276 277 279 280 281 283 284 286 287 289 290 291 293 294 296 297
 298 299 300 302 303 304 305 307 308 309 310

C:\Users\rhqud\AppData\Local\Temp/ipykernel_14828/3722576918.py:81: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Train_set_X = np.array(Train_set_X)
C:\Users\rhqud\AppData\Local\Temp/ipykernel_14828/3722576918.py:82: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Train_set_y = np.array(Train_set_y)
C:\Users\rhqud\AppData\Local\Temp/ipykernel_14828/3722576918.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you

In [27]:
Train_set_y[0]

array([ 0, 20, 19, 19,  0, 19, 24,  1,  2, 19,  2,  0,  2, 19,  0,  0, 19,
       19, 19, 19,  0, 19,  0,  0,  0,  3,  0,  0,  0,  2, 26,  2, 19,  0,
       19, 19, 19, 24,  2,  1, 19,  1,  0,  2, 19,  1, 19, 19, 19,  1, 19,
       19,  1, 23,  0, 19,  1,  0, 19,  0, 19, 19,  0,  1, 19, 19, 19,  0,
        2,  2,  1,  3, 19, 19, 19,  2,  1,  0, 19,  0, 19,  0,  0,  0, 19,
        0, 24, 24,  1, 19, 19, 19,  2,  0, 19, 19,  1, 24, 19, 19,  0,  0,
       19,  0,  0, 19,  6, 19,  0,  0,  0, 19,  0,  0,  0,  0,  1, 19,  1,
       19, 19,  0,  0, 19,  0, 19, 19, 19,  0,  2, 19,  0, 19, 19, 19, 19,
        0,  0,  0, 19, 19,  0,  0, 19, 19, 19, 19,  0,  0,  0, 19,  0, 19,
        1,  0,  1,  0, 19, 19,  0, 19,  1, 24, 19,  0, 19,  2,  0, 19,  0,
        0, 19, 19,  3, 21, 19, 19, 19,  0,  2,  4,  1, 19, 19,  0,  0,  0,
       19,  0,  3,  1,  0,  2, 19,  2,  0, 19,  0, 19,  0,  1, 19, 19, 19,
        0, 19,  0, 24, 19,  1,  0,  2,  1,  2, 19, 19,  0,  0, 19, 28, 19,
        0, 19,  1, 19,  0

In [28]:
trials=Trials()
#trials = pickle.load(open(path+site+'0929_suffle_onlystep4_KfoldCV+accuracy+weightx_t+0_nomalx_ECx- 2019x.p', "rb"))    #trials불러오기
starttime = time.time()
#if optype == "tpe":
best = fmin(XGB,space_xgb,algo=tpe.suggest,max_evals=100,trials=trials)
optime = time.time() - starttime
#else:
#  best = fmin(XGB,space_xgb,algo=rand.suggest,max_evals=10,trials=trials,rstate= np.random.RandomState(seed_value))

pickle.dump(trials, open(path+"\\models"+"\\xgb_histoy_%s.p" %(time.strftime('%Y%m%d_%H', time.localtime(starttime))), "wb"))     #trials저장하기
#print(best)
#print(space_eval(space_xgb, best))

###테스트셋에 테스트
###테스트셋에 테스트
learn=best['learn']
nesti=int(best['nesti'])
maxd=int(best['maxd'])
minc=int(best['minc'])
gamm=best['gamm']
subsa=best['subsa']
colsample_b=best['colsample_b']
learn=best['learn']
# scalepos=best['scalepos']

#
'''
learn=0.0107605633620193
nesti=int(100)
maxd=int(5)
minc=int(2)
gamm=22.5170863426934
subsa=0.315853538049625
colsample_b=0.801462768751276
scalepos=int(9)
'''
######최적화모델성능
xgb=XGBClassifier(
        learning_rate =learn,
        n_estimators=nesti,
        max_depth=maxd,
        min_child_weight=minc,
        gamma=gamm,
        subsample=subsa,
        colsample_bytree=colsample_b,
        nthread=-1,
        use_label_encoder=False,
        # scale_pos_weight=scalepos,
        n_jobs = -1,
        seed=100,
        # num_class=32,
        )


xgb.fit(x_prime_train, y_train)#,eval_set=eval_set)
#최종점수#
y_pred=xgb.predict(x_prime_test)
from sklearn import metrics
#결과
matricresult = metrics.classification_report(y_test, y_pred, digits=3)
pickle.dump(xgb, open(path+"\\models"+"\\xgb_model_%s.p" %(time.strftime('%Y%m%d_%H', time.localtime(starttime))), "wb"))
pickle.dump(matricresult, open(path+"\\models"+"\\xgb_matrix_%s.p" %(time.strftime('%Y%m%d_%H', time.localtime(starttime))), "wb"))


100%|██████████| 100/100 [3:40:08<00:00, 132.09s/trial, best loss: -0.4927369749001771]  


C:\Users\rhqud\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rhqud\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rhqud\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

In [29]:
answer = np.array([label_description[label_decoder[int(val)]] for val in y_test])
predss = np.array([label_description[label_decoder[int(val)]] for val in y_pred])

new_crosstab = pd.crosstab(answer, predss, rownames=['answer'], colnames=['preds'])
new_crosstab

preds,시설포도_정상,시설포도노균병_중기,시설포도탄저병_초기,일소피해_말기,파프리카_정상,파프리카흰가루병_말기,파프리카흰가루병_중기,파프리카흰가루병_초기
answer,,,,,,,,
시설포도_정상,36,0,0,0,1,0,0,2
시설포도노균병_중기,0,2,0,0,0,0,0,0
시설포도노균병_초기,2,0,0,0,0,0,0,0
시설포도탄저병_초기,1,0,4,0,0,0,0,0
일소피해_말기,1,0,0,0,0,0,0,0
일소피해_초기,1,0,0,0,0,0,0,0
축과병_초기,1,0,0,0,0,0,0,0
파프리카_정상,0,0,0,0,32,0,0,0
파프리카흰가루병_말기,0,0,0,1,3,1,1,0


In [30]:
from sklearn import metrics

# Print the confusion matrix
print(metrics.confusion_matrix(y_test, y_pred))

# Print the precision and recall, among other metrics
print(metrics.classification_report(y_test, y_pred, digits=3))

[[32  0  0  0  0  0  0  0  0  0  0]
 [ 2  5  1  0  0  0  0  0  0  0  0]
 [ 0  3  0  0  0  0  0  0  0  0  0]
 [ 3  0  1  1  0  0  0  0  0  1  0]
 [ 1  2  0  0 36  0  0  0  0  0  0]
 [ 0  0  0  0  1  4  0  0  0  0  0]
 [ 0  0  0  0  2  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  2  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  0  0]]
              precision    recall  f1-score   support

           0      0.842     1.000     0.914        32
           1      0.500     0.625     0.556         8
           2      0.000     0.000     0.000         3
           3      1.000     0.167     0.286         6
          19      0.857     0.923     0.889        39
          20      1.000     0.800     0.889         5
          23      0.000     0.000     0.000         2
          24      1.000     1.000     1.000         2
          26      0.000     0.000     0.000         1
          28      0.000     0.000     0.000         1
        

C:\Users\rhqud\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rhqud\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\rhqud\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo